# TODO: Move back to notebooks directory.

In [7]:
import globaligner as ga
import conclude
import start 
import itertools
import random
import sys
from typing import NamedTuple
from pprint import pprint
from pathlib import Path

In [8]:
alignment_results = ga.find_global_alignment(
    seq_1="CGGTCTTAGCATATGTTGGCATAC",
    seq_2="ATTAGCATCATAGTGGA",
    match_score=2,
    mismatch_score=-2,
    gap_open_score=-5,
    gap_extension_score=-2
)

In [9]:
ga.make_dp_array(
    seq_1="AGG",
    seq_2="GAG",
    costing_mat=alignment_results.costing_mat,
    max_cost=ga.get_max_val(alignment_results.costing_mat),
    gap_open_cost=alignment_results.gap_open_cost
)

[[(0, 0, 0), (15, 7, 15), (15, 10, 15), (15, 13, 15)],
 [(15, 15, 7), None, None, None],
 [(15, 15, 10), None, None, None],
 [(15, 15, 13), None, None, None]]

In [3]:
alignment_results.write(desc_1="rat", desc_2="mouse")

rat
mouse

CGGTCTTAGCAT-ATGTTGGCATAC
    *||||||| ||**|||*    
----ATTAGCATCATAGTGGA----

score: -14
cost: 55
###########################################
# Settings
###########################################
scoring_mat:
    A  C  G  T  -
A   2 -2 -2 -2 -2
C  -2  2 -2 -2 -2
G  -2 -2  2 -2 -2
T  -2 -2 -2  2 -2
-  -2 -2 -2 -2  2

gap_open_score: -5

costing_mat:
   A C G T -
A  0 4 4 4 3
C  4 0 4 4 3
G  4 4 0 4 3
T  4 4 4 0 3
-  3 3 3 3 0

gap_open_cost: 4



In [4]:
mat = {
    "laugh": {"laugh": 343, "smi": 3838, "a": 3},
    "smi": {"laugh": 2, "smi": 33, "a": 39834},
    "a": {"laugh": 35352, "smi": 4533, "a": 4}
}

nl = [
    [343, 3838],
    [2, 33]
]

In [5]:
conclude.pprint_mat(mat)

AttributeError: module 'conclude' has no attribute 'pprint_mat'

In [ ]:
conclude.print_nested_list_aligned(nl)

 343 3838
   2   33



In [ ]:
for j in range(5):
    for row in alignment_results.costing_mat.values():
        cell = row[j]

{'A': 0, 'C': 4, 'G': 4, 'T': 4, '-': 3}
{'A': 4, 'C': 0, 'G': 4, 'T': 4, '-': 3}
{'A': 4, 'C': 4, 'G': 0, 'T': 4, '-': 3}
{'A': 4, 'C': 4, 'G': 4, 'T': 0, '-': 3}
{'A': 3, 'C': 3, 'G': 3, 'T': 3, '-': 0}


In [ ]:
len(list(alignment_results.scoring_mat.values())[0].keys())

5

In [ ]:
b = "".join([f"hello {3 + 3}", f"{3 - 7} for yu"])
print(b)

hello 6-4 for yu


In [ ]:
a = [list(inner_dict.values()) for inner_dict in alignment_results.scoring_mat.values()]
conclude.print_nested_list_aligned(a)

  2 -2 -2 -2 -2
 -2  2 -2 -2 -2
 -2 -2  2 -2 -2
 -2 -2 -2  2 -2
 -2 -2 -2 -2  2



In [ ]:
%%timeit
a = "".join(alignment_printout_generator)

197 ns ± 77.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:
conclude.write_alignment(
    alignment_results=alignment_results,
    output="testing_me_again"
)

In [ ]:
conclude.print_alignment(alignment_results)

seq_1
seq_2

CGGTCTTAGCAT-ATGTTGGCATAC
    *||||||| ||**|||   | 
----ATTAGCATCATAGTGG---A-



In [ ]:
simple_scoring_settings = start.SimpleScoringSettings(*(None,)*4)
print(simple_scoring_settings)

match_score must be convertible to an integer.


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
scoring_settings = ga.ScoringSettings(*(None,)*5)
costing_settings = ga.CostingSettings(*(None,)*4)
alignment_settings = ga.AlignmentSettings(scoring_settings, costing_settings)
print(alignment_settings)

AttributeError: module 'globaligner' has no attribute 'ScoringSettings'

In [ ]:
alignment_settings.scoring_settings.scoring_mat

In [ ]:
ncbi_match_score = 2
ncbi_mismatch_score = -3
gap_open_cost = 4
ncbi_gap_extension_score = -2
scoring_mat = {
    "A": {"A": ncbi_match_score, "C": ncbi_mismatch_score, "G": ncbi_mismatch_score, "T": ncbi_mismatch_score, "-": ncbi_gap_extension_score},
    "C": {"A": ncbi_mismatch_score, "C": ncbi_match_score, "G": ncbi_mismatch_score, "T": ncbi_mismatch_score, "-": ncbi_gap_extension_score},
    "G": {"A": ncbi_mismatch_score, "C": ncbi_mismatch_score, "G": ncbi_match_score, "T": ncbi_mismatch_score, "-": ncbi_gap_extension_score},
    "T": {"A": ncbi_mismatch_score, "C": ncbi_mismatch_score, "G": ncbi_mismatch_score, "T": ncbi_match_score, "-": ncbi_gap_extension_score},
    "-": {"A": ncbi_gap_extension_score, "C": ncbi_gap_extension_score, "G": ncbi_gap_extension_score, "T": ncbi_gap_extension_score, "-": ncbi_match_score},
}
pprint(scoring_mat)

{'-': {'-': 2, 'A': -2, 'C': -2, 'G': -2, 'T': -2},
 'A': {'-': -2, 'A': 2, 'C': -3, 'G': -3, 'T': -3},
 'C': {'-': -2, 'A': -3, 'C': 2, 'G': -3, 'T': -3},
 'G': {'-': -2, 'A': -3, 'C': -3, 'G': 2, 'T': -3},
 'T': {'-': -2, 'A': -3, 'C': -3, 'G': -3, 'T': 2}}


In [ ]:
max_score = start.get_max_val(scoring_mat)
costing_mat = start.scoring_mat_to_costing_mat(
    scoring_mat=scoring_mat,
    max_score=max_score
)
max_cost = start.get_max_val(costing_mat)
costing_mat

{'A': {'A': 0, 'C': 5, 'G': 5, 'T': 5, '-': 3},
 'C': {'A': 5, 'C': 0, 'G': 5, 'T': 5, '-': 3},
 'G': {'A': 5, 'C': 5, 'G': 0, 'T': 5, '-': 3},
 'T': {'A': 5, 'C': 5, 'G': 5, 'T': 0, '-': 3},
 '-': {'A': 3, 'C': 3, 'G': 3, 'T': 3, '-': 0}}

In [ ]:
start.costing_mat_to_scoring_mat(
    costing_mat=costing_mat,
    max_score=max_score
)

{'A': {'A': 2, 'C': -3, 'G': -3, 'T': -3, '-': -2},
 'C': {'A': -3, 'C': 2, 'G': -3, 'T': -3, '-': -2},
 'G': {'A': -3, 'C': -3, 'G': 2, 'T': -3, '-': -2},
 'T': {'A': -3, 'C': -3, 'G': -3, 'T': 2, '-': -2},
 '-': {'A': -2, 'C': -2, 'G': -2, 'T': -2, '-': 2}}

In [ ]:
seq_1 = "GGAGGA"
seq_2 = "GAG"
dp_array = ga.make_dp_array(
    seq_1=seq_1,
    seq_2=seq_2,
    cost_mat=cost_mat,
    max_cost=max_cost,
    gap_open_cost=gap_open_cost
)


In [ ]:
dp_array

[[(0, 0, 0), (24, 3, 24), (24, 5, 24), (24, 7, 24)],
 [(24, 24, 4), None, None, None],
 [(24, 24, 7), None, None, None],
 [(24, 24, 10), None, None, None],
 [(24, 24, 13), None, None, None],
 [(24, 24, 16), None, None, None],
 [(24, 24, 19), None, None, None]]

In [ ]:
# Loop through the dp_array and write the
# best costs to get to each position.
ga.dp_array_forward(
    dp_array=dp_array,
    seq_1=seq_1,
    seq_2=seq_2,
    cost_mat=cost_mat,
    gap_open_cost=gap_open_cost
)


In [ ]:
dp_array

[[(0, 0, 0), (24, 3, 24), (24, 5, 24), (24, 7, 24)],
 [(24, 24, 4), (0, 7, 7), (6, 3, 9), (5, 5, 11)],
 [(24, 24, 7), (4, 10, 4), (3, 7, 7), (3, 6, 9)],
 [(24, 24, 10), (10, 13, 7), (4, 10, 7), (6, 7, 7)],
 [(24, 24, 13), (10, 16, 10), (10, 13, 8), (4, 11, 10)],
 [(24, 24, 16), (13, 19, 13), (13, 16, 11), (8, 14, 8)],
 [(24, 24, 19), (19, 22, 16), (13, 19, 14), (14, 16, 11)]]

In [ ]:
alignment_results_1 = ga.dp_array_backward(
    dp_array=dp_array,
    seq_1=seq_1,
    seq_2=seq_2,
    cost_mat=cost_mat,
    gap_open_cost=gap_open_cost
)

In [ ]:
alignment_results_1.cost

11

In [ ]:
ga.final_cost_to_score(alignment_results_1.cost, m=len(seq_1), n=len(seq_2), max_score=max_score)

-5

In [ ]:

alignment = ga.find_global_alignment(
    seq_1=seq_1,
    seq_2=seq_2,
    cost_mat=cost_mat,
    gap_open_cost=gap_open_cost
)
score = ga.final_cost_to_score(
    cost=alignment["cost"],
    m=len(seq_1),
    n=len(seq_2),
    max_score=max_score
)

ga.print_alignment(
    seq_1_aligned=alignment["seq_1_aligned"],
    mid=alignment["middle_part"],
    seq_2_aligned=alignment["seq_2_aligned"],
    score=score
)

In [ ]:
seq_1, seq_2 = ga.draw_two_random_seqs(
    alphabet=["A", "C", 'G', "T"],
    min_len_seq_1=2,
    width_required_seq_1=10,
    min_len_seq_2=2,
    width_required_seq_2=10,
    divergence=0.9
)

seq_1 = "GTAGGCGGTC"
seq_2 = "CAGCTGC"
print(seq_1)
print(seq_2)
seq_1_aligned_out, middle_part_out, seq_2_aligned_out, cost = ga.find_global_alignment(
    seq_1=seq_1,
    seq_2=seq_2,
    cost_mat=cost_mat,
    gap_open_cost=gap_open_cost
)
print(seq_1_aligned_out)
print(middle_part_out)
print(seq_2_aligned_out)
print(cost)
print(ga.final_cost_to_score(
    cost=cost,
    m=len(seq_1),
    n=len(seq_2),
    max_score=ncbi_match_score
))

In [ ]:
best_paths_mat = [
    [
        [10, 8, 4, 4],
        [0, 1, 8, 4],
        [0, 1, 1, 12],
        [0, 1, 1, 8],
        [0, 1, 1, 12],
        [0, 1, 1, 1],
        [0, 1, 1, 8]
    ],
    [
        [10, 4, 4, 4],
        [9, 5, 5, 5],
        [9, 9, 5, 5],
        [9, 0, 13, 9],
        [9, 0, 0, 13],
        [9, 0, 0, 9],
        [9, 0, 0, 0]
    ],
    [
        [10, 4, 4, 4],
        [0, 10, 7, 6],
        [0, 2, 11, 6],
        [0, 3, 2, 15],
        [0, 2, 3, 10],
        [0, 2, 11, 14],
        [0, 3, 10, 15]
    ]
]

ga.traceback_3(
    best_paths_mat=best_paths_mat,
    h_start=1,
    seq_1=seq_1,
    seq_2=seq_2
)

In [ ]:
print(ga.final_cost_to_score(
    cost=19,
    # m=len(seq_1),
    # n=len(seq_2),
    m=6,
    n=3,
    max_score=ncbi_match_score
))

In [ ]:
ga.final_score_to_cost(
    score=-14,
    m=len(seq_1),
    n=len(seq_2),
    max_score=ncbi_match_score
)

In [ ]:
middle_row_index = len(seq_1)
best_paths_mat = ga.init_best_paths_matrix(
    dynamic_prog_num_rows=middle_row_index + 1,
    dynamic_prog_num_cols=len(seq_2) + 1
)
best_paths_mat

In [ ]:
dynamic_prog_num_cols = len(seq_2) + 1
partial_dp_mat = ga.init_partial_dynamic_prog_matrix_2(
    seq_1=seq_1,
    seq_2=seq_2,
    cost_mat=cost_mat,
    gap_open_cost=gap_open_cost,
    dynamic_prog_num_cols=dynamic_prog_num_cols
)
partial_dp_mat

In [ ]:
moves_for_gap_open_penalty_from_left = {0, 3, 4, 11, 12, 14, 19, 22, 23}
moves_for_gap_open_penalty_from_up = {0, 1, 2, 9, 10, 13, 19, 20, 21}
situation_mapper={
    # from_diag_best_path_type = 0
    # from_left_best_path_type == 1
    # and from_up_best_path_type == 3
    # 3-way ties
    ((0, 0, 0), (0, 1, 3)): 15,
    # 2-way ties for low
    ((0, 0, 2), (0, 1, 3)): 9,
    ((0, 2, 0), (0, 1, 3)): 11,
    ((2, 0, 0), (0, 1, 3)): 5,
    # 2-way ties for high
    ((0, 1, 1), (0, 1, 3)): 0,
    ((1, 0, 1), (0, 1, 3)): 1,
    ((1, 1, 0), (0, 1, 3)): 3,
    # no ties
    ((0, 1, 2), (0, 1, 3)): 0,
    ((1, 0, 2), (0, 1, 3)): 1,
    ((1, 2, 0), (0, 1, 3)): 3,
    ((0, 2, 1), (0, 1, 3)): 0,
    ((2, 0, 1), (0, 1, 3)): 1,
    ((2, 1, 0), (0, 1, 3)): 3,
    # from_left_best_path_type == 1
    # and from_up_best_path_type == 4
    # 3-way ties
    ((0, 0, 0), (0, 1, 4)): 16,
    # 2-way ties for low
    ((0, 0, 2), (0, 1, 4)): 9,
    ((0, 2, 0), (0, 1, 4)): 12,
    ((2, 0, 0), (0, 1, 4)): 6,
    # 2-way ties for high
    ((0, 1, 1), (0, 1, 4)): 0,
    ((1, 0, 1), (0, 1, 4)): 1,
    ((1, 1, 0), (0, 1, 4)): 4,
    # no ties
    ((0, 1, 2), (0, 1, 4)): 0,
    ((1, 0, 2), (0, 1, 4)): 1,
    ((1, 2, 0), (0, 1, 4)): 4,
    ((0, 2, 1), (0, 1, 4)): 0,
    ((2, 0, 1), (0, 1, 4)): 1,
    ((2, 1, 0), (0, 1, 4)): 4,
    # from_left_best_path_type == 2
    # and from_up_best_path_type == 3
    # 3-way ties
    ((0, 0, 0), (0, 2, 3)): 17,
    # 2-way ties for low
    ((0, 0, 2), (0, 2, 3)): 10,
    ((0, 2, 0), (0, 2, 3)): 11,
    ((2, 0, 0), (0, 2, 3)): 7,
    # 2-way ties for high
    ((0, 1, 1), (0, 2, 3)): 0,
    ((1, 0, 1), (0, 2, 3)): 2,
    ((1, 1, 0), (0, 2, 3)): 3,
    # no ties
    ((0, 1, 2), (0, 2, 3)): 0,
    ((1, 0, 2), (0, 2, 3)): 2,
    ((1, 2, 0), (0, 2, 3)): 3,
    ((0, 2, 1), (0, 2, 3)): 0,
    ((2, 0, 1), (0, 2, 3)): 2,
    ((2, 1, 0), (0, 2, 3)): 3,
    # from_left_best_path_type == 2
    # and from_up_best_path_type == 4
    # 3-way ties
    ((0, 0, 0), (0, 2, 4)): 18,
    # 2-way ties for low
    ((0, 0, 2), (0, 2, 4)): 10,
    ((0, 2, 0), (0, 2, 4)): 12,
    ((2, 0, 0), (0, 2, 4)): 8,
    # 2-way ties for high
    ((0, 1, 1), (0, 2, 4)): 0,
    ((1, 0, 1), (0, 2, 4)): 2,
    ((1, 1, 0), (0, 2, 4)): 4,
    # no ties
    ((0, 1, 2), (0, 2, 4)): 0,
    ((1, 0, 2), (0, 2, 4)): 2,
    ((1, 2, 0), (0, 2, 4)): 4,
    ((0, 2, 1), (0, 2, 4)): 0,
    ((2, 0, 1), (0, 2, 4)): 2,
    ((2, 1, 0), (0, 2, 4)): 4,
    # from_diag_best_path_type = 19
    # from_left_best_path_type == 1
    # and from_up_best_path_type == 3
    # 3-way ties
    ((0, 0, 0), (19, 1, 3)): 24,
    # 2-way ties for low
    ((0, 0, 2), (19, 1, 3)): 20,
    ((0, 2, 0), (19, 1, 3)): 22,
    ((2, 0, 0), (19, 1, 3)): 5,
    # 2-way ties for high
    ((0, 1, 1), (19, 1, 3)): 19,
    ((1, 0, 1), (19, 1, 3)): 1,
    ((1, 1, 0), (19, 1, 3)): 3,
    # no ties
    ((0, 1, 2), (19, 1, 3)): 19,
    ((1, 0, 2), (19, 1, 3)): 1,
    ((1, 2, 0), (19, 1, 3)): 3,
    ((0, 2, 1), (19, 1, 3)): 19,
    ((2, 0, 1), (19, 1, 3)): 1,
    ((2, 1, 0), (19, 1, 3)): 3,
    # from_left_best_path_type == 1
    # and from_up_best_path_type == 4
    # 3-way ties
    ((0, 0, 0), (19, 1, 4)): 25,
    # 2-way ties for low
    ((0, 0, 2), (19, 1, 4)): 20,
    ((0, 2, 0), (19, 1, 4)): 23,
    ((2, 0, 0), (19, 1, 4)): 6,
    # 2-way ties for high
    ((0, 1, 1), (19, 1, 4)): 19,
    ((1, 0, 1), (19, 1, 4)): 1,
    ((1, 1, 0), (19, 1, 4)): 4,
    # no ties
    ((0, 1, 2), (19, 1, 4)): 19,
    ((1, 0, 2), (19, 1, 4)): 1,
    ((1, 2, 0), (19, 1, 4)): 4,
    ((0, 2, 1), (19, 1, 4)): 19,
    ((2, 0, 1), (19, 1, 4)): 1,
    ((2, 1, 0), (19, 1, 4)): 4,
    # from_left_best_path_type == 2
    # and from_up_best_path_type == 3
    # 3-way ties
    ((0, 0, 0), (19, 2, 3)): 26,
    # 2-way ties for low
    ((0, 0, 2), (19, 2, 3)): 21,
    ((0, 2, 0), (19, 2, 3)): 22,
    ((2, 0, 0), (19, 2, 3)): 7,
    # 2-way ties for high
    ((0, 1, 1), (19, 2, 3)): 19,
    ((1, 0, 1), (19, 2, 3)): 2,
    ((1, 1, 0), (19, 2, 3)): 3,
    # no ties
    ((0, 1, 2), (19, 2, 3)): 19,
    ((1, 0, 2), (19, 2, 3)): 2,
    ((1, 2, 0), (19, 2, 3)): 3,
    ((0, 2, 1), (19, 2, 3)): 19,
    ((2, 0, 1), (19, 2, 3)): 2,
    ((2, 1, 0), (19, 2, 3)): 3,
    # from_left_best_path_type == 2
    # and from_up_best_path_type == 4
    # 3-way ties
    ((0, 0, 0), (19, 2, 4)): 27,
    # 2-way ties for low
    ((0, 0, 2), (19, 2, 4)): 21,
    ((0, 2, 0), (19, 2, 4)): 23,
    ((2, 0, 0), (19, 2, 4)): 8,
    # 2-way ties for high
    ((0, 1, 1), (19, 2, 4)): 19,
    ((1, 0, 1), (19, 2, 4)): 2,
    ((1, 1, 0), (19, 2, 4)): 4,
    # no ties
    ((0, 1, 2), (19, 2, 4)): 19,
    ((1, 0, 2), (19, 2, 4)): 2,
    ((1, 2, 0), (19, 2, 4)): 4,
    ((0, 2, 1), (19, 2, 4)): 19,
    ((2, 0, 1), (19, 2, 4)): 2,
    ((2, 1, 0), (19, 2, 4)): 4
}

In [ ]:
partial_dp_mat, cur_cell_best_cum_cost, best_paths_mat = ga.warmup_align_2(
    seq_1=seq_1,
    seq_2=seq_2,
    best_paths_mat=best_paths_mat,
    partial_dp_mat=partial_dp_mat,
    gap_open_cost=gap_open_cost,
    cost_mat=cost_mat,
    moves_for_gap_open_penalty_from_left=moves_for_gap_open_penalty_from_left,
    moves_for_gap_open_penalty_from_up=moves_for_gap_open_penalty_from_up,
    situation_mapper=situation_mapper
)

In [ ]:
partial_dp_mat

In [ ]:
partial_dp_mat, cur_cell_best_cum_cost, best_paths_mat = ga.do_core_align_2(
    seq_1=seq_1,
    seq_2=seq_2,
    best_paths_mat=best_paths_mat,
    partial_dp_mat=partial_dp_mat,
    gap_open_cost=gap_open_cost,
    cost_mat=cost_mat,
    moves_for_gap_open_penalty_from_left=moves_for_gap_open_penalty_from_left,
    moves_for_gap_open_penalty_from_up=moves_for_gap_open_penalty_from_up,
    situation_mapper=situation_mapper
)

In [ ]:
print(best_paths_mat)

In [ ]:
partial_dp_mat

In [ ]:
cur_cell_best_cum_cost

In [ ]:
ga.final_cost_to_score(
    cost=cur_cell_best_cum_cost,
    m=len(seq_1),
    n=len(seq_2),
    max_score=ncbi_match_score
)

In [ ]:
ga.final_score_to_cost(
    score=-23,
    m=len(seq_1),
    n=len(seq_2),
    max_score=ncbi_match_score
)

In [ ]:
best_paths_mat

In [ ]:
partial_dp_mat

In [ ]:
seq_1_aligned, middle_part, seq_2_aligned = ga.traceback_2(
    best_paths_mat=best_paths_mat,
    seq_1=seq_1,
    seq_2=seq_2, 
)

print(seq_1_aligned)
print(middle_part)
print(seq_2_aligned)

In [ ]:
# seq_1 = "CATGGG"
# seq_1 = "C"
# seq_2 = "ACTG"
# seq_2 = "TATT"
# seq_1 = "ACACAACTAGTGCTACGTAT"
# seq_2 = "T"
# seq_1 = "TC"
# seq_2 = "T"
# seq_1 = "GTCAGCAT"
# seq_2 = "CTCTGAACACG"
# seq_1 = "CGCCTC"
# seq_2 = "GTCG"
# seq_1 = "CGCCT"
# seq_2 = "GTCG"
# seq_1 = "CATGGG"
# seq_2 = "ACTG"
dynamic_prog_num_rows = len(seq_1) + 1
dynamic_prog_num_cols = len(seq_2) + 1

In [ ]:
alignment = ga.align(
    seq_1=seq_1,
    seq_2=seq_2,
    scoring_mat=scoring_mat,
    gap_existence_cost=gap_existence_cost
)
ga.print_alignment(
    *alignment,
    chars_per_line=70
)

In [ ]:
best_paths_mat = [[1, 1, 1, 1, 1], [2, 0, 1, 0, 0], [2, 0, 0, 1, 0], [2, 0, 0, 0, 1], [2, 2, 0, 0, 0], [2, 2, 0, 2, 0], [2, 2, 2, 0, 2]]
best_paths_mat

In [ ]:
alignment = ga.traceback(
    best_paths_mat=best_paths_mat,
    seq_1=seq_1,
    seq_2=seq_2
)
ga.print_alignment(*alignment)